In [ ]:

import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt


(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

x_train, x_test = x_train / 255.0, x_test / 255.0


data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1)
])


mlp_model = models.Sequential([
    layers.Flatten(input_shape=(32, 32, 3)),
    layers.Dense(512, activation='relu'),
    layers.Dense(256, activation='relu'),
    layers.Dense(10, activation='softmax')
])

mlp_model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

print("\nTraining MLP model...")
mlp_history = mlp_model.fit(x_train, y_train,
                            validation_data=(x_test, y_test),
                            epochs=10, batch_size=128, verbose=1)


cnn_model = models.Sequential([
    data_augmentation,

    layers.Conv2D(32, (3,3), activation='relu', input_shape=(32, 32, 3)),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')
])

cnn_model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

print("\nTraining CNN model...")
cnn_history = cnn_model.fit(x_train, y_train,
                            validation_data=(x_test, y_test),
                            epochs=10, batch_size=128, verbose=1)


def plot_metric(histories, metric, title):
    plt.figure(figsize=(10, 5))
    for name, history in histories.items():
        plt.plot(history.history[metric], label=f'{name} Train')
        plt.plot(history.history[f'val_{metric}'], '--', label=f'{name} Val')
    plt.title(title)
    plt.xlabel('Epochs')
    plt.ylabel(metric.capitalize())
    plt.legend()
    plt.grid(True)
    plt.show()


plot_metric({'MLP': mlp_history, 'CNN': cnn_history},
             'accuracy', 'Training and Validation Accuracy Comparison')
plot_metric({'MLP': mlp_history, 'CNN': cnn_history},
             'loss', 'Training and Validation Loss Comparison')


mlp_acc = mlp_model.evaluate(x_test, y_test, verbose=0)[1]
cnn_acc = cnn_model.evaluate(x_test, y_test, verbose=0)[1]

print(f"\nFinal Test Accuracy:")
print(f"MLP: {mlp_acc:.4f}")
print(f"CNN: {cnn_acc:.4f}")